# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install langchain

In [2]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os

from google.colab import userdata

OPENAI_API_KEY = userdata.get('openaikey')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [8]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [10]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2, openai_api_key=userdata.get('openaikey'))


In [11]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [12]:
# old code
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [13]:
product = "Rubber ducks"
chain.invoke(product)

AIMessage(content='Introducing the ultimate bath time companion - the Rubber Duck! This quacky little friend is perfect for adding some fun to your daily routine. With its waterproof design and adorable appearance, this rubber duck is sure to make a splash in your tub. Say goodbye to boring baths and hello to endless entertainment with the Rubber Duck!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 27, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d5e8a1c9-b1ed-4eb8-a163-1f7d5ec485f4-0', usage_metadata={'input_tokens': 27, 'output_tokens': 65, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

## SimpleSequentialChain

In [14]:
from langchain.chains import SimpleSequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9, openai_api_key=userdata.get('openaikey'))

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

<ipython-input-15-bd508ec08e52>:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [16]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [18]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Introducing the Quacktastic Rubber Duck! This little guy may be small, but he packs a punch in the bath time fun department. Made of durable rubber, he's perfect for splish-splashing around in the tub. Plus, his adorable design is sure to bring a smile to your face. Say goodbye to boring bath times and hello to the quacktastic life with this rubber duck!
I would rate this product a 9 out of 10. It is made of durable rubber and has an adorable design that brings joy to bath time. It adds a fun element to the bath and is sure to entertain children.

> Finished chain.


{'input': 'Rubber ducks',
 'output': 'I would rate this product a 9 out of 10. It is made of durable rubber and has an adorable design that brings joy to bath time. It adds a fun element to the bath and is sure to entertain children.'}

**Repeat the above twice for different products**

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.9, openai_api_key=userdata.get('openaikey'))


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='fr_review' #Give a name to your output
                    )


In [21]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key='fr_short_review' #give a name to this output
                    )


In [22]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [23]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [24]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [25]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Goût moyen, mousse instable, possible contrefaçon ou lot périmé.',
 'en_short_review': 'Average taste, unstable foam, possible counterfeit or expired batch.',
 'rating': 'I would rate the quality of the translation as an 8 out of 10. The translation accurately conveys the original meaning of the phrase and maintains the same tone and context.'}

**Repeat the above twice for different products or reviews**

In [26]:
review_2 = df.Review[0]
overall_chain.invoke(review_2)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.',
 'fr_short_review': 'Ensemble taille king, service client excellent, draps frais.',
 'en_short_review': 'Ensemble king size, excellent customer service, fresh sheets.',
 'rating': 'I would rate the translation a 9 out of 10. It effectively conveys the meaning of the original sentence and captures the key elements of the message. The choice of words and structure is clear and easy to understand.'}

In [27]:
review_3 = df.Review[6]
overall_chain.invoke(review_3)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'fr_short_review': 'Belle, rapide, fonctionnelle, durabilité à vérifier, achetée récemment.',
 'en_short_review': 'Beautiful, fast, functional, durability to be checked, recently purchased.',
 'rating': 'I would rate the translation a 9 out of 10. The translation accurately conveys the meaning of the original text and maintains the key descriptors of the product. The only improvement could be to make the phrase "durabilité à vérifier" more fluid, but overall, it is a high-quality translation.'}

## Router Chain

In [28]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [29]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [30]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [31]:
llm = ChatOpenAI(temperature=0, openai_api_key=userdata.get('openaikey'))

In [32]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [42]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts. If none of the candidates seem appropriate, or if you are unsure, choose "DEFAULT".
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [43]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

#router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True
                        )

In [44]:
router_chain.invoke("Explain what principle component analysis is using linear algebra")

{'input': 'Explain what principle component analysis is using linear algebra',
 'destination': 'computer science',
 'next_inputs': {'input': 'Explain what principle component analysis is using linear algebra'}}

In [45]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")




> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.


In [46]:
response

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in machine learning and data analysis to reduce the dimensionality of a dataset while preserving as much variance as possible. \n\nIn linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector.\n\nTo perform PCA, we first center the data by subtracting the mean of each feature from the data points. Then, we compute the covariance matrix of the centered data. Next, we find the eigenvectors and eigenvalues of the covariance matrix. \n\nThe eigenvectors form a new basis for the data, where the first eigenvector corresponds to the direction of maximum variance, the second eigenvector corresponds to the direction of second maximum variance, a

In [47]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."}

In [48]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': '2 + 2 is equal to 4.'}

In [51]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [52]:
chain.invoke("What is the purpose of DAGs")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the purpose of Directed Acyclic Graphs (DAGs)'}
> Finished chain.


{'input': 'What is the purpose of Directed Acyclic Graphs (DAGs)',
 'text': 'Directed Acyclic Graphs (DAGs) are a type of graph data structure that have a specific direction for their edges and do not contain any cycles. They are commonly used in computer science and mathematics for various purposes, such as representing dependencies between tasks, scheduling algorithms, and optimizing algorithms.\n\nThe main purpose of DAGs is to model relationships between objects or tasks in a way that allows for efficient processing and analysis. Because DAGs do not contain cycles, they can be used to represent a sequence of events or tasks that must be completed in a specific order without any circular dependencies.\n\nIn computer science, DAGs are often used in algorithms for tasks such as topological sorting, shortest path algorithms, and dynamic programming. They provide a way to efficiently represent and solve complex problems by breaking them down into smaller, more manageable subproblems.\n\

In [53]:
chain.invoke("What was the most sginificant event of 1453?")



> Entering new MultiPromptChain chain...
History: {'input': 'What was the most significant event of 1453?'}
> Finished chain.


{'input': 'What was the most significant event of 1453?',
 'text': 'The most significant event of 1453 was the fall of Constantinople to the Ottoman Empire. This event marked the end of the Byzantine Empire, which had been a powerful and influential force in the region for over a thousand years. The fall of Constantinople had far-reaching consequences, including the spread of Ottoman power throughout the Eastern Mediterranean and the Balkans, the displacement of Greek scholars and artists, and the disruption of trade routes between Europe and Asia. It also had a significant impact on the development of Western Europe, as it prompted many scholars and artists to flee to Italy, helping to spark the Renaissance.'}

In [55]:
chain.invoke("What is the most succesful team in Euroleague championship?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the most succesful team in Euroleague championship?'}
> Finished chain.


{'input': 'What is the most succesful team in Euroleague championship?',
 'text': 'Real Madrid is the most successful team in Euroleague championship history, having won the title a record 10 times.'}